# Data Aggregation

## HTML Parsing

Read the HTML source from the web:

In [1]:
import requests

url = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2018.html'
r = requests.get(url)
print(r.text[:100])  # print(r.text) to retrieve the full source

<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">
    
    <!-- Lev


Parse the source to retrieve the exam schedule:

In [2]:
from bs4 import BeautifulSoup

html = BeautifulSoup(r.text, 'html.parser')
tbody = html.find('tbody')
exam_schedule = {}

for tr in tbody.find_all('tr'):
    tds  = tr.find_all('td')
    meet = tds[0].string.strip()
    day  = tds[1].string.strip()
    date = tds[2].string.strip()
    time = tds[3].string.strip()
    exam_schedule[meet] = (day, date, time)

for k, v in exam_schedule.items():
    print('%10s: %s' % (k, v))

 08:00 MWF: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
08:00 TTHF: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
  08:30 MW: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
 08:30 TTH: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
 09:00 MWF: ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
09:00 TThF: ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
  10:00 MW: ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
 10:00 MWF: ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
 10:00 TTH: ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
10:00 TThF: ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
 11:00 MWF: ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
11:00 TThF: ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
  11:30 MW: ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
 11:30 TTh: ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
 12:00 MWF: ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
12:00 TThF: ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
   1:00 MW: ('Thursday', '3-May', '3:00 P.M - 5:30 P.M')
  1:00 MWF: ('Thursday', '3-May', '3:00 

## Regular Expressions

Retrieve only rows with time and day:

In [3]:
import re

TIME_DAY = re.compile('(\d{1,2}:\d\d)\s+([A-Za-z]+)')

m = TIME_DAY.match('8:00 MW')
print('Time: %5s, Day: %s' % (m.group(1), m.group(2)))

m = TIME_DAY.match('12:30 TThF')
print('Time: %5s, Day: %s' % (m.group(1), m.group(2)))

Time:  8:00, Day: MW
Time: 12:30, Day: TThF


In [4]:
m = TIME_DAY.match('Math*')
print(m)

None


Handle different ways of indicating time:

In [5]:
TIME = re.compile('(\d{1,2}):(\d\d)\s*([AaPp]\.?\s*[Mm]\.?)?')

times = ['8:00', '12:30', '2:30pm', '2:30 pm', '2:30PM', '2:30P.M', '2:30P.M.', '2:30 PM.']

for t in times:
    m = TIME.match(t)
    print('%10s -> (%2s, %2s, %s)' % (t, m.group(1), m.group(2), m.group(3)))

      8:00 -> ( 8, 00, None)
     12:30 -> (12, 30, None)
    2:30pm -> ( 2, 30, pm)
   2:30 pm -> ( 2, 30, pm)
    2:30PM -> ( 2, 30, PM)
   2:30P.M -> ( 2, 30, P.M)
  2:30P.M. -> ( 2, 30, P.M.)
  2:30 PM. -> ( 2, 30, PM.)


## Normalization

Convert the matched result to military time:

In [6]:
def norm_time(hour, minute, period):
    h = int(hour)
    m = int(minute)
    
    if period:
        p = period[0].upper()
        if p == 'P': h += 12

    return h * 100 + m

In [7]:
for t in times:
    m = TIME.match(t)
    n = norm_time(m.group(1), m.group(2), m.group(3))
    print('%10s -> %4d' % (t, n))

      8:00 ->  800
     12:30 -> 1230
    2:30pm -> 1430
   2:30 pm -> 1430
    2:30PM -> 1430
   2:30P.M -> 1430
  2:30P.M. -> 1430
  2:30 PM. -> 1430


Convert days into a binary form:

In [8]:
def norm_days(days):
    DAYS = [('M', 0), ('TU', 1), ('W', 2), ('TH', 3), ('F', 4)]
    days = days.upper()
    b = ['0'] * 5

    for d, i in DAYS:
        if d in days:
            b[i] = '1'
            days = days.replace(d, '')

    if 'T' in days:
        b[1] = '1'
        days = days.replace('T', '')

    return int(''.join(b), 2)

In [9]:
days = ['MWF', 'TuTh', 'MTuWThF', 'TThF', 'MWFf']

for d in days:
    n = norm_days(d)
    print('%7s %5s %2d' % (d, bin(n)[2:], n))

    MWF 10101 21
   TuTh  1010 10
MTuWThF 11111 31
   TThF  1011 11
   MWFf 10101 21


In [10]:
## Exam Schedule Extraction

In [11]:
def extract_exam_schedule(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html.parser')
    tbody = html.find('tbody')
    schedule = {}

    for tr in tbody.find_all('tr'):
        tds = tr.find_all('td')
        t = tds[0].string.split()

        if len(t) == 2:
            time = days = None
            m = TIME.match(t[0])
            if m: time = norm_time(m.group(1), m.group(2), m.group(3))
            days = norm_days(t[1])

            if time and days:
                if time < 700: time += 1200
                key  = (time, days)
                day  = tds[1].string.strip()
                date = tds[2].string.strip()
                time = tds[3].string.strip()
                schedule[key] = (day, date, time)

    return schedule

In [12]:
url = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2018.html'
exam_schedule = extract_exam_schedule(url)
for k, v in exam_schedule.items():
    print('%10s: %s' % (k, v))

 (800, 21): ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
 (800, 11): ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
 (830, 20): ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
 (830, 10): ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
 (900, 21): ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
 (900, 11): ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
(1000, 20): ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
(1000, 21): ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
(1000, 10): ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
(1000, 11): ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
(1100, 21): ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
(1100, 11): ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
(1130, 20): ('Friday', '4-May', '8:00 A.M - 10:30 A.M')
(1130, 10): ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
(1200, 21): ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
(1200, 11): ('Friday', '4-May', '3:00 P.M - 5:30 P.M')
(1300, 20): ('Thursday', '3-May', '3:00 P.M - 5:30 P.M')
(1300, 21): ('Thursday', '3-May', '3:00 

## HTTP Session

The full content of a dynamically generated  webpage sometimes cannot be retrieved:

In [13]:
url = 'http://atlas.college.emory.edu/schedules/index.php?select=QTM'
r = requests.get(url)
print(r.text[:100])  # print(r.text) to retrieve the full source


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xht


Open a session and retrieve the root page:

In [14]:
url = 'http://atlas.college.emory.edu/schedules/index.php'
s = requests.session()
r = s.get(url, params={'t': '5181'})
print(r.url)

http://atlas.college.emory.edu/schedules/index.php?t=5181


Retrieve the dynamic webpage:

In [15]:
r = s.get(url, params={'select': 'QTM'})
print(r.text[:100])  # print(r.text) to retrieve the full source


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xht


## Course Information Extraction

Helper methods to extract course information:

In [16]:
def course_title(section):
    a = section.find('a')
    t = a.text.split()
    return t[0], ' '.join(t[1:])

def course_days(section):
    days = [day.string.strip() for day in section.find_all('span', {'class': 'day'})]
    return ''.join(days) if days else '-'

def course_time(section):
    time = section.find('div', {'class': 'time'}).string
    if time:
        t = TIME.findall(time)
        return norm_time(*t[0]), norm_time(*t[1])
    else:
        return -1, -1

def course_location(section):
    loc = section.find('div', {'class': 'location'}).string
    return loc if loc else '-'

def course_instructors(section):
    its = [inst.string.strip() for inst in section.find_all('div', {'class': 'instructor'}) if inst.string]
    return '; '.join(its).strip() if its else '-'

Parse the HTML source:

In [17]:
def extract_course_info(term_id):
    url = 'http://atlas.college.emory.edu/schedules/index.php'
    s = requests.session()
    s.get(url, params={'t': term_id})
    programs = ['QTM']
    course_info = {}

    for program in programs:
        r = s.get(url, params={'select': program})
        html = BeautifulSoup(r.text, 'html.parser')
        for course in html.find_all('div', {'class': 'course'}):
            course_number = course.find('h3').string
            for section in course.find_all('li', {'class': 'section'}):
                section_id, title = course_title(section)
                days = norm_days(course_days(section))
                time = course_time(section)
                location = course_location(section)
                instructors = course_instructors(section)
                course_info[(course_number, section_id)] = (title, days, time, location, instructors)

    return course_info

In [18]:
course_info = extract_course_info('5181')
for k, v in course_info.items():
    print('%8s-%2s: %s' % (k[0], k[1], v))

 QTM 100- 1: ('Intro to Stat Inference', 20, (830, 945), 'Math & Science Center - E208', 'Chen, Xuan')
 QTM 100-11: ('Intro to Stat Inference', 1, (900, 950), 'Callaway Center C101', 'Chen, Xuan; Morin, Elyse')
 QTM 100-12: ('Intro to Stat Inference', 1, (1100, 1150), 'Math & Science Center - N306', 'Park, Kevin; Chen, Xuan')
 QTM 100-13: ('Intro to Stat Inference', 1, (1300, 1350), 'Math & Science Center - N306', 'Park, Kevin; Chen, Xuan')
 QTM 100-14: ('Intro to Stat Inference', 1, (1000, 1050), 'Math & Science Center - N306', 'Zaretski, Aliaksandr; Chen, Xuan')
 QTM 100-15: ('Intro to Stat Inference', 1, (2400, 2450), 'Callaway Center C101', 'Lauer, Jillian; Chen, Xuan')
 QTM 100-16: ('Intro to Stat Inference', 1, (1400, 1450), 'Math & Science Center - N306', 'Buller, Ian; Chen, Xuan')
 QTM 100-17: ('Intro to Stat Inference', 1, (1000, 1050), 'New Psyc Bldg 230 (36 Eagle Ro', 'Kim, Ho Jin; Pollak, Rebecca')
 QTM 100-18: ('Intro to Stat Inference', 1, (2400, 2450), 'New Psyc Bldg 230

## Data Aggregation

Integrate the exam schedule into each section of the course:

In [19]:
url = 'http://registrar.emory.edu/faculty-staff/exam-schedule/spring-2018.html'
exam_schedule = extract_exam_schedule(url)
course_info = extract_course_info('5181')
course_exam = {}

for k, v in course_info.items():
    days = v[1]
    time = v[2]
    key = (time[0], days)

    if key in exam_schedule:
        course_exam['-'.join(k)] = exam_schedule[key]

In [20]:
for k, v in course_exam.items():
    print(k, v)

QTM 100-1 ('Tuesday', '8-May', '11:30 A.M - 2:00 P.M')
QTM 100-2 ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
QTM 100-3 ('Tuesday', '8-May', '3:00 P.M - 5:30 P.M')
QTM 100-4 ('Wednesday', '9-May', '11:30 A.M - 2:00 P.M')
QTM 110-1 ('Thursday', '3-May', '3:00 P.M - 5:30 P.M')
QTM 120-1 ('Monday', '7-May', '8:00 A.M - 10:30 A.M')
QTM 210-1 ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
QTM 220-1 ('Monday', '7-May', '3:00 P.M - 5:30 P.M')
QTM 285-1 ('Tuesday', '8-May', '8:00 A.M - 10:30 A.M')
QTM 302W-1 ('Wednesday', '9-May', '11:30 A.M - 2:00 P.M')
QTM 345-1 ('Tuesday', '8-May', '6:30 P.M - 9:00 P.M')
QTM 355-1 ('Thursday', '3-May', '3:00 P.M - 5:30 P.M')
QTM 385-1 ('Wednesday', '9-May', '3:00 P.M - 5:30 P.M')
QTM 385-3 ('Thursday', '3-May', '8:00 A.M - 10:30 A.M')
QTM 491-1 ('Friday', '4-May', '11:30 A.M - 2:00 P.M')


Save the aggregated data to a JSON file:

In [21]:
import json

jsonfile = '../dat/course_exam_spring_2018.json'
with open(jsonfile, 'w') as fout:
    json.dump(course_exam, fout)

Load the saved data from the JSON file:

In [22]:
with open(jsonfile) as fin:
    d = json.load(fin)

for k, v in d.items():
    print(k, v)

QTM 100-1 ['Tuesday', '8-May', '11:30 A.M - 2:00 P.M']
QTM 100-2 ['Monday', '7-May', '8:00 A.M - 10:30 A.M']
QTM 100-3 ['Tuesday', '8-May', '3:00 P.M - 5:30 P.M']
QTM 100-4 ['Wednesday', '9-May', '11:30 A.M - 2:00 P.M']
QTM 110-1 ['Thursday', '3-May', '3:00 P.M - 5:30 P.M']
QTM 120-1 ['Monday', '7-May', '8:00 A.M - 10:30 A.M']
QTM 210-1 ['Thursday', '3-May', '8:00 A.M - 10:30 A.M']
QTM 220-1 ['Monday', '7-May', '3:00 P.M - 5:30 P.M']
QTM 285-1 ['Tuesday', '8-May', '8:00 A.M - 10:30 A.M']
QTM 302W-1 ['Wednesday', '9-May', '11:30 A.M - 2:00 P.M']
QTM 345-1 ['Tuesday', '8-May', '6:30 P.M - 9:00 P.M']
QTM 355-1 ['Thursday', '3-May', '3:00 P.M - 5:30 P.M']
QTM 385-1 ['Wednesday', '9-May', '3:00 P.M - 5:30 P.M']
QTM 385-3 ['Thursday', '3-May', '8:00 A.M - 10:30 A.M']
QTM 491-1 ['Friday', '4-May', '11:30 A.M - 2:00 P.M']
